In [1]:
import json
import os

from elasticsearch import Elasticsearch

In [2]:
HOST = os.environ.get("HOST")
USER = os.environ.get("USER")
PASSWORD = os.environ.get("PASSWORD")

In [3]:
def elasticsearch_connection():
    es = Elasticsearch([{'host':HOST,'port':9200}], http_auth=(USER, PASSWORD))
    return es

es = elasticsearch_connection()
es.ping()

True

In [9]:
query = {
  "size": 0, 
  "query": {
    "bool": {
      "must": [
        {"exists": {"field": "subtema"}}
      ],
      "must_not": [
        {"term": {
          "subtema.keyword": "reset"
        }},
        {"term": {
          "tema.keyword": "reset"
        }},
        {"term": {
          "workspace_name.keyword": "reset"
        }},
        {"term": {
          "department.keyword": "reset"
        }}
      ]
    }
  },
  "aggs": {
    "cadeias": {
      "terms": {
        "field": "workspace_name.keyword",
        "size": 10
      },
      "aggs": {
        "temas": {
          "terms": {
            "field": "tema.keyword",
            "size": 10
          },
          "aggs": {
            "subtemas": {
              "terms": {
                "field": "subtema.keyword",
                "size": 10
              }
            }
          }
        }
      }
    }
  }
}

In [10]:
index = 'prd2-yara-omniv2_peculiardata'

In [11]:
res = es.search(index=index, body=query)

In [12]:
data = res["aggregations"]

In [13]:
with open("../data.json", "w") as f:
    json.dump(data, f, ensure_ascii=False)